# Import Modules

In [1]:
import pandas as pd
import numpy as np
import os #for managing files
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline


import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Dropout, Flatten, MaxPooling2D

# Load The Dataset

In [2]:
TRAIN_DIR = '../input/facial-expression-dataset/train/train/'
TEST_DIR = '../input/facial-expression-dataset/test/test/'

In [3]:
def load_dataset(directory):
    image_paths = []
    labels = []
    
    for label in os.listdir(directory):
        for filename in os.listdir(directory+label):
            image_path = os.path.join(directory,label,filename)
            image_paths.append(image_path)
            labels.append(label)
            
        print(label,"Completed")
    return image_paths, labels

In [4]:
#convert into dataframe
train = pd.DataFrame()
train['image'], train['label'] = load_dataset(TRAIN_DIR)
#shuffle the dataset
train = train.sample(frac = 1).reset_index(drop = True)
train.head()

In [5]:
#convert into dataframe
test = pd.DataFrame()
test['image'], test['label'] = load_dataset(TEST_DIR)
test.head()

## Exploratory Data Analysis

In [6]:
sns.countplot(train['label'])

In [7]:
from PIL import Image
img = Image.open(train['image'][0])
plt.imshow(img, cmap= 'gray');

In [8]:
#to display grid of images
plt.figure(figsize =(16,16))
files = train.iloc[0:25]

for index, file, label in files.itertuples():
    plt.subplot(5,5,index+1)
    img = load_img(file)
    plt.imshow(img)
    plt.title(label)
    plt.axis('off')

# Feature Extraction

In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale = True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [10]:
train_features = extract_features(train['image'])

In [11]:
test_features = extract_features(test['image'])

In [12]:
#normalize the image

x_train = train_features/255.0
x_test = test_features/255.0

In [13]:
##convert label to integer
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [14]:
y_train = to_categorical(y_train,num_classes= 7)
y_test = to_categorical(y_test,num_classes= 7)

In [15]:
y_train[0]

In [16]:
#configure
input_shape = (48,48,1)
output_class= 7

# Model Creation

In [17]:
model = Sequential()

#convolutional layers
model.add(Conv2D(128,kernel_size = (3,3), activation ='relu',input_shape = input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size = (3,3), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size = (3,3), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size = (3,3), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

#fully connected layers
model.add(Dense(512,activation ='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation ='relu'))
model.add(Dropout(0.3))

#output layer
model.add(Dense(output_class, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics = 'accuracy')


In [ ]:
#train the model
history = model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test,y_test))

# Plot The Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs,acc,'b',label='Training Accuracy')
plt.plot(epochs, val_acc,'r',label ='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()

plt.figure()
loss = history.history['accuracy']
val_acc = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs,acc,'b',label='Training loss')
plt.plot(epochs, val_acc,'r',label ='Validation loss')
plt.title('loss Graph')
plt.legend()

plt.show()

# Test with Image Data

In [ ]:
image_index = random.randint(0, len(test))
print("Original Output: ",test['label'][image_index])
pred = model.predict(x_test[image_index].reshape(1,48,48,1))
prediction_label = le.inverse_transform([pred.argmax()])[0]
print("Prediction Output: ",prediction_label)
plt.imshow(x_test[image_index].reshape(48,48),cmap='gray')